In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import PIL
from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, TensorDataset
import copy

import os

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
imsize = 64

In [ ]:
loader = transforms.Compose([
    transforms.Resize([imsize,imsize]),  # scale imported image
    transforms.ToTensor()])

In [ ]:
def image_loader(image_name):
    image = Image.open(image_name)
    if('.png' in image_name):
        image = image.convert("RGB")
    # fake batch dimension required to fit network's input dimensions
    image = loader(image).unsqueeze(0)
    return image.to(torch.float)

In [ ]:
setu_path = '/home/shuzhou/Documents/dihiggs/image/setu'
nonsetu_path = '/home/shuzhou/Documents/dihiggs/image/non-set'

In [ ]:
def create_dataset(path, imsize, label):
    file_list = os.listdir(path)
    file_list = [x for x in file_list if('.jpg' in x or '.png' in x)]
    file_num = len(file_list)
    im_tensor = torch.zeros((file_num, 3, imsize, imsize))
    label_tensor = torch.ones((file_num))*label
    i=0
    for file in file_list:
        im_tensor[i,] = image_loader(path+'/'+file)
        i=i+1
        print("image {0} is loaded".format(i))
    return(im_tensor, label_tensor)
    

In [ ]:
PIL.Image.MAX_IMAGE_PIXELS = None
setu_tensor, setu_label = create_dataset(setu_path, imsize,1.0)

In [ ]:
print(setu_label)

In [ ]:
unloader = transforms.ToPILImage()  # reconvert into PIL image

plt.ion()

def imshow(tensor, title=None):
    image = tensor.cpu().clone()  # we clone the tensor to not do changes on it
    image = image.squeeze(0)      # remove the fake batch dimension
    image = unloader(image)
    plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

In [ ]:
imshow(setu_tensor[8,])

In [ ]:
nonsetu_tensor, nonsetu_label = create_dataset(nonsetu_path, imsize, label=0)

In [ ]:
train_tensor = torch.cat((setu_tensor,nonsetu_tensor),axis=0)

In [ ]:
print(train_tensor.size())

In [ ]:
train_label = torch.cat((setu_label,nonsetu_label),axis=0)
print(train_label.size())

In [ ]:
trainData = TensorDataset(train_tensor, train_label)
trainLoader = DataLoader(trainData, batch_size=16,shuffle=True, num_workers=4,pin_memory=True)

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [ ]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.layer5 = self._make_layer(block, 512, num_blocks[4], stride=2)
        self.linear = nn.Linear(512*block.expansion, 1)
        self.out_act = nn.Sigmoid()

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        out = self.out_act(out)
        return out
        
        
        
        


In [ ]:
def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3,3])

In [ ]:
model = ResNet50().to(device)

In [ ]:
score = model(setu_tensor[0:5,])
print(score.size())

In [ ]:
print(score)

In [ ]:
def train(model, device, train_loader, loss_function, optimizer, epoch, num_class, use_apex = False):
    model.train()
    average_loss=0
    batch_n=0
    for batch_idx, (data, target) in enumerate(train_loader):
        batch_n=batch_n+1
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        data=data.squeeze()
        output = model(data)
        #my_weight=sample_weight/sample_weight.mean()
        if(num_class == 2):
            output=output.reshape(-1)
            loss = loss_function(output, target)
            #loss = custom_loss(output, target,my_weight)
            if(use_apex == False):
                loss.backward()
            average_loss=average_loss+loss.item()
        else:
            target = target.long()
            loss = loss_function(output, target)
            loss = loss
            loss = loss.mean()
            if(use_apex == False):
                loss.backward()
            else:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            average_loss=average_loss+loss.item()
        optimizer.step()
        torch.cuda.empty_cache()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    print("Average loss in epoch ",epoch," is: ",average_loss/batch_n)
    return(average_loss/batch_n)

In [ ]:
optimizer = optim.Adam(model.parameters())
loss_function = nn.BCELoss().cuda()

In [ ]:
for i in range(0,20):
    train(model, device, trainLoader, loss_function, optimizer,i,2)

In [ ]:
def train_model(model, trainLoader, num_class, patient, max_epoch, model_name):
    i = 0
    min_loss = 999
    best_epoch = 0
    counter = 0
    for i in range(0,max_epoch):
        loss = train(model, device, trainLoader, loss_function, optimizer,i,num_class)
        if(loss<min_loss):
            min_loss = loss
            counter = 0
            best_epoch = i
            torch.save(model.state_dict(),'training-temp.pt')
        else:
            counter = counter+1
        if(counter == patient):
            print("Stopping, best epoch is: ",best_epoch," lowest loss is: ",min_loss)
            model.load_state_dict(torch.load('training-temp.pt'))
            torch.save(model.state_dict(),model_name)
            break
    print("Stopping at epoch {0}".format(max_epoch))
    model.load_state_dict(torch.load('training-temp.pt'))
    torch.save(model.state_dict(),model_name)
    
            
    

In [ ]:
train_model(model, trainLoader, 2, 4, 40, 'setu-resnet.pt')

In [ ]:
def setu_score(setu_name, model_name):
    model = ResNet50().to(device)
    model.load_state_dict(torch.load(model_name))
    tensor = image_loader(setu_name)
    model.eval()
    #tensor = tensor.unsqueeze(0)
    with torch.no_grad():
        tensor = tensor.to(device)
        score = model(tensor)
        score = score.cpu().numpy()
    return(score)

In [ ]:
setu_path = '/home/shuzhou/Documents/dihiggs/image/setu'
nonsetu_path = '/home/shuzhou/Documents/dihiggs/image/non-set'

In [ ]:
import random

In [ ]:
setu_list = os.listdir(setu_path)
setu_list = [x for x in setu_list if('.jpg' in x or '.png' in x)]

In [ ]:
setu = random.choice(setu_list)
print(setu)

In [ ]:
score = setu_score(setu_path+'/'+setu, 'setu-resnet.pt')
print(score)

In [ ]:
test = nonsetu_tensor[1:5,].to(device)
score = model(test)

In [ ]:
print(score.size())
score1 = score.unsqueeze(0)
print(score1.size())

In [ ]:
imshow(nonsetu_tensor[1,])